# Import librairies

In [256]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium

In [257]:
from modules.connecting_to_bdd import fetch_table_from_trafic_db
from modules.geodata import convert_to_linestring, has_close_event

# Import dataset

In [258]:
evenements_df = fetch_table_from_trafic_db('events_data')

In [259]:
trafic_df = fetch_table_from_trafic_db('trafic_routier')

In [260]:
trafic_df.head()

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}"
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,"{47.23083661767159,-1.5704468983375661}","{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}"
2,497-20250218T205200,497,180,224,2.0,3,Lauriol I4,Fluide,37,22,"{47.23531301184315,-1.5635261418937887}","{{-1.563539784805616,47.236320687488636},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5635261418937887,47.23531301184315}"
3,5444-20250218T205200,5444,240,177,2.9,3,Pasteur P1,Fluide,40,16,"{47.21255561611825,-1.579953397082114}","{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.579953397082114,47.21255561611825}"
4,5053-20250218T205200,5053,60,224,0.8,3,Barbusse P2,Fluide,44,18,"{47.22705355840811,-1.5524880999813486}","{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:52:00,Point,"{-1.5524880999813486,47.22705355840811}"


In [261]:
trafic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554761 entries, 0 to 1554760
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id_technique       1554761 non-null  object        
 1   id                 1554761 non-null  int64         
 2   debit              1554761 non-null  int64         
 3   longueur           1554761 non-null  int64         
 4   taux_occupation    1554761 non-null  float64       
 5   code_couleur       1554761 non-null  int64         
 6   nom_du_troncon     1554761 non-null  object        
 7   etat_du_trafic     1554761 non-null  object        
 8   temps_de_parcours  1554761 non-null  int64         
 9   vitesse            1554761 non-null  int64         
 10  geo_point_2d       1554761 non-null  object        
 11  geometrie          1554761 non-null  object        
 12  shape_geo          1554761 non-null  object        
 13  horodatage         1554761 

In [262]:
evenements_df.head()

,name,description,startdate,enddate,location_name,address,city,postalcode,location_uid,coordinates_geo,event_id
0,Football : FC Nantes / Paris SG,Match - 29e journée - Ligue 1 - Saison 2024/2025,2025-04-13 17:00:00,2025-04-13 19:00:00,Stade de la Beaujoire Louis Fonteneau,"330 Route de Saint Joseph, Nantes",Nantes,44300,95545941,"{-1.527812, 47.25867}",926afe1f-3bcc-4cc1-a2bd-3597dd36d86d
1,Football : FC Nantes / Toulouse FC,Match - 31e journée - Ligue 1 - Saison 2024/2025,2025-04-27 17:00:00,2025-04-27 19:00:00,Stade de la Beaujoire Louis Fonteneau,"330 Route de Saint Joseph, Nantes",Nantes,44300,95545941,"{-1.527812, 47.25867}",19bb79da-1c20-41a2-a53d-c45bcae5a0ce
2,Blandine Lehout - La Vie de ta mère,Blandine Lehout - La Vie de ta mère,2025-10-01 20:00:00,2025-10-01 22:30:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",da51b672-08bc-49e6-bcd6-516090c48e77
3,Alexandre Kominek - Bâtard sensible,Alexandre Kominek - Bâtard sensible,2025-03-01 20:30:00,2025-03-01 22:00:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",209c09e0-8d45-4429-ab15-7e1bf979f9c1
4,Comédie Le Clan des divorcées,Comédie Le Clan des divorcées,2025-03-02 15:00:00,2025-03-02 16:40:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",2eae861f-2e7f-4ff4-8c77-8f72b719fb23


# Visualisation

In [263]:
# Convertir les donnée en type LineString
trafic_df['geometrie_linestring'] = trafic_df['geometrie'].apply(convert_to_linestring)

In [264]:
# Convertir la colonne 'coordinates_geo' en objets Point
evenements_df['geometrie_point'] = evenements_df['coordinates_geo'].apply(lambda x: Point(eval(x)))  

In [265]:
# Convertir les DataFrames en GeoDataFrames
trafic_gdf = gpd.GeoDataFrame(trafic_df, geometry='geometrie_linestring', crs="EPSG:4326")  # Système de coordonnées WGS 84
evenements_gdf = gpd.GeoDataFrame(evenements_df, geometry='geometrie_point', crs="EPSG:4326")

In [266]:
# Faire un dataframe avec les troncon_unique
unique_troncon = trafic_gdf.drop_duplicates(subset='nom_du_troncon', keep='first')

In [267]:
# Faire un dataframe avec les events unique
unique_event_place = evenements_gdf.drop_duplicates(subset='location_name', keep='first')

In [268]:
unique_event_place['location_name'].value_counts()

location_name
Stade de la Beaujoire Louis Fonteneau    1
Cité internationale des Congrès          1
Palais des Sports Beaulieu               1
Parc des Expositions de la Beaujoire     1
STEREOLUX                                1
Zénith de Nantes Métropole               1
Name: count, dtype: int64

In [269]:
unique_event_place.head()

,name,description,startdate,enddate,location_name,address,city,postalcode,location_uid,coordinates_geo,event_id,geometrie_point
0,Football : FC Nantes / Paris SG,Match - 29e journée - Ligue 1 - Saison 2024/2025,2025-04-13 17:00:00,2025-04-13 19:00:00,Stade de la Beaujoire Louis Fonteneau,"330 Route de Saint Joseph, Nantes",Nantes,44300,95545941,"{-1.527812, 47.25867}",926afe1f-3bcc-4cc1-a2bd-3597dd36d86d,POINT (47.25867 -1.52781)
2,Blandine Lehout - La Vie de ta mère,Blandine Lehout - La Vie de ta mère,2025-10-01 20:00:00,2025-10-01 22:30:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",da51b672-08bc-49e6-bcd6-516090c48e77,POINT (47.21331 -1.54406)
48,Handball : HBC Nantes / C' Chartres MH,dans la H Arena,2025-03-01 20:30:00,2025-03-01 22:00:00,Palais des Sports Beaulieu,"5 Rue André Tardieu, Nantes",Nantes,44200,57179132,"{-1.53728, 47.208987}",3ceea7d4-7b4b-4299-9c3b-42cdbd85b146,POINT (47.20899 -1.53728)
49,Foire de Nantes,Foire de Nantes 2025,2025-04-03 10:00:00,2025-04-07 18:00:00,Parc des Expositions de la Beaujoire,"2B Rue des Pays de la Loire, Nantes",Nantes,44300,34310089,"{-1.529241, 47.258884}",2435290b-90d1-40ee-89c7-32350ce297df,POINT (47.25888 -1.52924)
50,"Black Country, New Road",Les Londonien·nes dévoilent un son oscillant e...,2025-10-10 20:00:00,2025-10-10 23:00:00,STEREOLUX,"4, Boulevard Léon Bureau, Nantes",Nantes,44200,21547859,"{-1.564217, 47.206315}",bb651100-9c7a-4aa8-a1d5-6c06a2f39fda,POINT (47.20632 -1.56422)


In [270]:
unique_troncon.head()

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo,geometrie_linestring
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}","LINESTRING (-1.58312 47.23519, -1.57863 47.23311)"
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,"{47.23083661767159,-1.5704468983375661}","{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}","LINESTRING (-1.56859 47.23188, -1.5723 47.22979)"
2,497-20250218T205200,497,180,224,2.0,3,Lauriol I4,Fluide,37,22,"{47.23531301184315,-1.5635261418937887}","{{-1.563539784805616,47.236320687488636},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5635261418937887,47.23531301184315}","LINESTRING (-1.56354 47.23632, -1.56351 47.23431)"
3,5444-20250218T205200,5444,240,177,2.9,3,Pasteur P1,Fluide,40,16,"{47.21255561611825,-1.579953397082114}","{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.579953397082114,47.21255561611825}","LINESTRING (-1.58103 47.21287, -1.57887 47.21225)"
4,5053-20250218T205200,5053,60,224,0.8,3,Barbusse P2,Fluide,44,18,"{47.22705355840811,-1.5524880999813486}","{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:52:00,Point,"{-1.5524880999813486,47.22705355840811}","LINESTRING (-1.55155 47.2276, -1.55213 47.2271..."


In [271]:
# Créer une carte centrée sur Nantes (coordonnées de Nantes)
map_nantes = folium.Map(location=[47.2184, -1.5536], zoom_start=13)

# # Ajouter tous les tronçons de trafic sous forme de lignes (LineString)
for idx, row in unique_troncon.iterrows():
    # S'assurer que la géométrie est un LineString
    if isinstance(row['geometrie_linestring'], LineString):
        # Convertir les coordonnées du LineString pour Folium
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometrie_linestring'].coords], 
            color='blue',  # Couleur de la ligne
            weight=2.5,  # Épaisseur de la ligne
            opacity=1
        ).add_to(map_nantes)

# Ajouter tous les événements sous forme de points (Point)
for idx, event in unique_event_place.iterrows():
    # S'assurer que la géométrie de l'événement est de type Point
    event_geom = event['geometrie_point']
    
    if isinstance(event_geom, Point):
        # Ajouter un point (Marker) pour chaque événement directement
        folium.Marker(
            location=[event_geom.x, event_geom.y],
            popup=event['location_name'],  
            icon=folium.Icon(color='red')  
        ).add_to(map_nantes)

# Afficher la carte
map_nantes
